## Weather forecasting Bangladesh using LSTM(StackLSTM)

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# **Step 1 : Load the Data**

In [ ]:
df= pd.read_csv('/kaggle/input/bangladesh-historical-weather-dataset-2008-2023/Bangladesh Historical Weather Dataset 2008-2023.csv')
df.head()

In [ ]:
df = pd.DataFrame(list(df['temperature_2m_mean (°C)']), index=df['time'], columns=['temp'])

In [ ]:
#data

In [ ]:
# To use data for 365 days
#df=data[:365]

In [ ]:
df.shape

# **Step 2 : Preprocessing the Data**

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(axis=0)

In [ ]:
df.shape

In [ ]:
df=df['temp'].values

In [ ]:
df[:5]

In [ ]:
df=df.reshape(-1,1)

In [ ]:
plt.figure(figsize=(25, 7))
plt.plot(df, linewidth=1)
# plt.grid()
plt.title("Time Series (temperature month wise)")
plt.show()

# **Step 3 : Scaling Data**

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df)

In [ ]:
data_scaled[:5]

In [ ]:
data_scaled.shape

In [ ]:
steps= 20
inp = []
out = []
for i in range(len(data_scaled) - (steps)):
    inp.append(data_scaled[i:i+steps])
    out.append(data_scaled[i+steps])

In [ ]:
out[:10]

reshape 

In [ ]:
import numpy as np

In [ ]:
inp= np.asanyarray(inp)
out= np.asanyarray(out)

# **Step 4 : Split the dataset into training and testing sets**

In [ ]:
len(df)*0.80

In [ ]:
x_train = inp[:4586,:,:]
x_test = inp[4586:,:,:]    
y_train = out[:4586]    
y_test= out[4586:]

In [ ]:
inp.shape

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Dense,RepeatVector, LSTM, Dropout
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model

# **Step 5: Model Building**

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences= True, input_shape= (20,1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model = Sequential()
model.add(LSTM(50, return_sequences= True, input_shape= (20,1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

# **Step 6: Training**

In [ ]:
model.fit(x_train,y_train,epochs=300, verbose=1, )

In [ ]:
model.evaluate(x_test, y_test)

# **Step 7: Predictions and Model evaluation**

Predictions on training set

In [ ]:
print("Predicted Value",model.predict(x_train)[10][0])
print("Expected value",y_train[10][0])

Predictions on testing set

In [ ]:
predictions=model.predict(x_test)

In [ ]:
print("Predicted Value",predictions[2][0])
print("Expected Value",y_test[2][0])

In [ ]:
predictions.shape

In [ ]:
y_test.shape

In [ ]:
pred_df=pd.DataFrame(predictions)

In [ ]:
pred_df['TrueValues']=y_test

In [ ]:
pred_df_new = pred_df.rename(columns={ 0: 'Predictions'})

In [ ]:
pred_df_new

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data= pred_df_new)
plt.title("Predictions VS True Values on Testing Set")

# **Step 8: Calculate the evaluation metrices**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Make predictions
y_pred = model.predict(x_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

# Print the results
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# **Step 9: Weather Predictions for next 30 Days**

In [ ]:
data_scaled.shape

In [ ]:
x_input=data_scaled[:20]

In [ ]:
x_input.shape

In [ ]:
# now reshaping the data
x_input = x_input.reshape(1, -1)

In [ ]:
x_input.shape

In [ ]:
# x_input = x_input.reshape((1, 20,1))
# yhat = model.predict(x_input, verbose=0)

In [ ]:
temp_input = list(x_input)
temp_input = temp_input[0].tolist()

In [ ]:
# demonstrate prediction
from numpy import array

lst_output=[]
n_steps=20
i=0
while(i<365):
    
    if(len(temp_input)>20):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
#         print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new = np.arange(1,5734)
day_pred = np.arange(5734,6099)

In [ ]:
plt.figure(figsize=(20,16))
plt.plot(day_new,data_scaled)
plt.plot(day_pred,lst_output)

In [ ]:
data_scaled[:5]

In [ ]:
df3 = data_scaled.tolist()
df3.extend(lst_output)
plt.figure(figsize=(12,8))
plt.plot(df3)

In [ ]:
# Calculate last 365 days
last_year_days = day_new[-365:]

# Extract data for last year
last_year_data = data_scaled[-365:]

# Extract forecast data
forecast_data = lst_output

# Plot last year data and forecast
plt.figure(figsize=(12, 8))
plt.plot(last_year_days, last_year_data, label="Actual Data")
plt.plot(day_pred, forecast_data, label="Predicted Data")

# Add labels and legend
plt.xlabel("Date")
plt.ylabel("Scaled Values")
plt.legend()
plt.title("Last Year's Data and Forecast")

# Show plot
plt.show()

In [ ]:
df3 = data_scaled[-365:].tolist()
df3.extend(lst_output)
plt.figure(figsize=(12,8))
plt.plot(df3)

In [ ]:
# # Check lengths of arrays
# print("Length of last_year_days:", len(last_year_days))
# print("Length of last_year_data_original:", len(last_year_data_original))
# print("Length of day_pred:", len(day_pred))
# print("Length of forecast_data_original:", len(forecast_data_original))

In [ ]:
# Calculate last 365 days
last_year_days = day_new[-365:]

# Extract data for last year
last_year_data = data_scaled[-365:]

# Extract forecast data
forecast_data = lst_output

# Inverse transform the scaled data to the original scale
last_year_data_original = scaler.inverse_transform(np.array(last_year_data).reshape(-1, 1))
forecast_data_original = scaler.inverse_transform(np.array(forecast_data).reshape(-1, 1))

# Print actual vs predicted values
actual_vs_predicted_df = pd.DataFrame({
    'Date': last_year_days,
    'Actual': last_year_data_original.flatten(),
    'Predicted': forecast_data_original.flatten()
})

print("Actual vs Predicted Values:")
print(actual_vs_predicted_df)

# **Rainfall**

In [ ]:
df_rainfall = pd.read_csv('/kaggle/input/bangladesh-historical-weather-dataset-2008-2023/Bangladesh Historical Weather Dataset 2008-2023.csv')
df_rainfall.head()

In [ ]:
df_rainfall = pd.DataFrame(list(df_rainfall['rain_sum (mm)']), index=df_rainfall['time'], columns=['rain'])

In [ ]:
df_rainfall.head()

In [ ]:
df_rainfall.isnull().sum()

In [ ]:
df_rainfall=df_rainfall.dropna(axis=0)
df_rainfall.isnull().sum()

In [ ]:
df_rainfall.shape

In [ ]:
df_rainfall=df_rainfall['rain'].values

In [ ]:
df_rainfall[:5]

In [ ]:
df_rainfall=df_rainfall.reshape(-1,1)

In [ ]:
plt.figure(figsize=(25, 7))
plt.plot(df_rainfall, linewidth=1)
# plt.grid()
plt.title("Time Series (rainfall month wise)")
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df_rainfall)

In [ ]:
data_scaled[:5]

In [ ]:
data_scaled.shape

In [ ]:
steps= 20
inp = []
out = []
for i in range(len(data_scaled) - (steps)):
    inp.append(data_scaled[i:i+steps])
    out.append(data_scaled[i+steps])
    
out[:10]

In [ ]:
inp= np.asanyarray(inp)
out= np.asanyarray(out)

In [ ]:
len(df_rainfall)*0.80

In [ ]:
x_train_rainfall = inp[:4586,:,:]
x_test_rainfall = inp[4586:,:,:]    
y_train_rainfall = out[:4586]    
y_test_rainfall = out[4586:]

In [ ]:
print(x_train_rainfall.shape, x_test_rainfall.shape)

In [ ]:
model_rainfall = Sequential()
model_rainfall.add(LSTM(50, return_sequences= True, input_shape= (20,1)))
model_rainfall.add(LSTM(50, return_sequences=True))
model_rainfall.add(LSTM(50))
model_rainfall.add(Dense(1))
model_rainfall.compile(loss = 'mean_squared_error', optimizer = 'adam')
model_rainfall = Sequential()
model_rainfall.add(LSTM(50, return_sequences= True, input_shape= (20,1)))
model_rainfall.add(LSTM(50, return_sequences=True))
model_rainfall.add(LSTM(50))
model_rainfall.add(Dense(1))
model_rainfall.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
model_rainfall.fit(x_train_rainfall,y_train_rainfall,epochs=300, verbose=1, )

In [ ]:
model_rainfall.evaluate(x_test_rainfall, y_test_rainfall)

In [ ]:
print("Predicted Value",model_rainfall.predict(x_train_rainfall)[10][0])
print("Expected value",y_train_rainfall[10][0])

In [ ]:
predictions_rainfall=model_rainfall.predict(x_test_rainfall)

In [ ]:
pred_df_rainfall=pd.DataFrame(predictions_rainfall)

In [ ]:
pred_df_rainfall['TrueValues']=y_test_rainfall

In [ ]:
pred_df_new_rainfall = pred_df_rainfall.rename(columns={ 0: 'Predictions_rainfall'})

In [ ]:
pred_df_new_rainfall

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data= pred_df_new_rainfall)
plt.title("Predictions VS True Values on Testing Set")

In [ ]:
# Make predictions
y_pred_rainfall = model_rainfall.predict(x_test_rainfall)

# Calculate MSE
mse = mean_squared_error(y_test_rainfall, y_pred_rainfall)

# Calculate MAE
mae = mean_absolute_error(y_test_rainfall, y_pred_rainfall)

# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test_rainfall, y_pred_rainfall) * 100

# Print the results
print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

In [ ]:
x_input_rainfall=data_scaled[:20]
x_input_rainfall.shape

In [ ]:
# now reshaping the data
x_input_rainfall = x_input_rainfall.reshape(1, -1)
x_input_rainfall.shape

In [ ]:
temp_input_rainfall = list(x_input_rainfall)
temp_input_rainfall = temp_input_rainfall[0].tolist()

In [ ]:
lst_output_rainfall=[]
n_steps=20
i=0
while(i<365):
    
    if(len(temp_input_rainfall)>20):
        #print(temp_input)
        x_input_rainfall=np.array(temp_input_rainfall[1:])
        print("{} day input {}".format(i,x_input_rainfall))
        x_input_rainfall=x_input_rainfall.reshape(1,-1)
        x_input_rainfall = x_input_rainfall.reshape((1, n_steps, 1))
        #print(x_input)
        yhat_rainfall = model_rainfall.predict(x_input_rainfall, verbose=0)
        print("{} day output {}".format(i,yhat_rainfall))
        temp_input_rainfall.extend(yhat_rainfall[0].tolist())
        temp_input_rainfall=temp_input_rainfall[1:]
        #print(temp_input)
        lst_output_rainfall.extend(yhat_rainfall.tolist())
        i=i+1
    else:
        x_input_rainfall = x_input_rainfall.reshape((1, n_steps,1))
        yhat_rainfall = model_rainfall.predict(x_input_rainfall, verbose=0)
        print(yhat_rainfall[0])
        temp_input_rainfall.extend(yhat_rainfall[0].tolist())
#         print(len(temp_input))
        lst_output_rainfall.extend(yhat_rainfall.tolist())
        i=i+1
    

print(lst_output_rainfall)

In [ ]:
day_new_rainfall = np.arange(1,5734)
day_pred_rainfall = np.arange(5734,6099)

In [ ]:
plt.figure(figsize=(20,16))
plt.plot(day_new_rainfall,data_scaled)
plt.plot(day_pred_rainfall,lst_output_rainfall)

In [ ]:
data_scaled[:5]

In [ ]:
df3_rainfall = data_scaled.tolist()
df3_rainfall.extend(lst_output_rainfall)
plt.figure(figsize=(12,8))
plt.plot(df3_rainfall)

In [ ]:
# Check lengths of arrays
print("Length of last_year_days:", len(last_year_days))
print("Length of last_year_data_original:", len(last_year_data_original))
print("Length of day_pred:", len(day_pred))
print("Length of forecast_data_original:", len(forecast_data_original))

In [ ]:
# Calculate last 365 days
last_year_days = day_new_rainfall[-365:]

# Extract data for last year
last_year_data = data_scaled[-365:]

# Extract forecast data
forecast_data = lst_output_rainfall

# Inverse transform the scaled data to the original scale
last_year_data_original = scaler.inverse_transform(np.array(last_year_data).reshape(-1, 1))
forecast_data_original = scaler.inverse_transform(np.array(forecast_data).reshape(-1, 1))

# Print actual vs predicted values
actual_vs_predicted_df = pd.DataFrame({
    'Date': last_year_days,
    'Actual': last_year_data_original.flatten(),
    'Predicted': forecast_data_original.flatten()
})

print("Actual vs Predicted Values:")
print(actual_vs_predicted_df)

# **Thanks You**